In [21]:
# grettings to : https://github.com/PacktPublishing/Learn-OpenAI-Whisper/blob/main/Chapter07/LOAIW_ch07_2_Quantizing_Distil_Whisper_with_OpenVINO.ipynb

## Prerequisites

Before diving into the tutorial, ensure that you have the necessary prerequisites in place. This includes authenticating with the Hugging Face Hub using your token and verifying the authentication by running the provided code cells. These steps are crucial for accessing the required models and datasets throughout the tutorial.



In [1]:
!pip install --upgrade huggingface_hub


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# Verify authentication
from huggingface_hub import whoami
whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

{'type': 'user',
 'id': '676187706ce06148154eb2f8',
 'name': 'aaiche',
 'fullname': 'aaiche',
 'isPro': False,
 'avatarUrl': '/avatars/1bf3857a4d2a4e5e6fb5381ba1ed7752.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'AA_TOKEN',
   'role': 'fineGrained',
   'createdAt': '2025-01-11T15:38:55.885Z',
   'fineGrained': {'canReadGatedRepos': False,
    'global': [],
    'scoped': [{'entity': {'_id': '676187706ce06148154eb2f8',
       'type': 'user',
       'name': 'aaiche'},
      'permissions': ['repo.content.read']}]}}}}

In [ ]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio]

## Loading the PyTorch model

Loading the PyTorch Whisper model is a straightforward process using the transformers library. The `AutoModelForSpeechSeq2Seq.from_pretrained` method is employed to initialize the model. In this tutorial, we will use the `distil-whisper/distil-large-v2` model as the default example. Please note that the model will be downloaded during the first run, which may take some time.

However, you have the flexibility to choose from a variety of models available in the [Distil-Whisper Hugging Face collection](https://huggingface.co/collections/distil-whisper/distil-whisper-models-65411987e6727569748d2eb6). Some alternative options include `distil-whisper/distil-medium.en` and `distil-whisper/distil-small.en`. Additionally, models of the original Whisper architecture are also accessible, which you can explore further [here](https://huggingface.co/openai).

It's important to highlight the significance of preprocessing and post-processing in the model's usage. The `AutoProcessor` class, specifically the `WhisperProcessor`, plays a crucial role in preparing the audio input data for the model. It handles tasks such as converting the audio to a Mel-spectrogram representation and decoding the predicted output token IDs back into a string using the tokenizer.

To ensure a smooth and efficient workflow, the `AutoProcessor` class streamlines the preprocessing and post-processing steps, allowing you to focus on the core functionality of the Whisper model. By leveraging this class, you can easily integrate the Whisper model into your speech recognition pipeline, regardless of the specific model variant you choose.

In [6]:
import ipywidgets as widgets

model_ids = {
    "Distil-Whisper": [
        "distil-whisper/distil-large-v2",
        "distil-whisper/distil-medium.en",
        "distil-whisper/distil-small.en"
    ],
    "Whisper": [
        "openai/whisper-large-v3",
        "openai/whisper-large-v2",
        "openai/whisper-large",
        "openai/whisper-medium",
        "openai/whisper-small",
        "openai/whisper-base",
        "openai/whisper-tiny",
        "openai/whisper-medium.en",
        "openai/whisper-small.en",
        "openai/whisper-base.en",
        "openai/whisper-tiny.en",
    ]
}

model_type = widgets.Dropdown(
    options=model_ids.keys(),
    value="Whisper",
    description="Model type:",
    disabled=False,
)

model_type

Dropdown(description='Model type:', index=1, options=('Distil-Whisper', 'Whisper'), value='Whisper')

In [7]:
model_id = widgets.Dropdown(
    options=model_ids[model_type.value],
    value=model_ids[model_type.value][1],
    description="Model:",
    disabled=False,
)

model_id

Dropdown(description='Model:', index=1, options=('openai/whisper-large-v3', 'openai/whisper-large-v2', 'openai…

## Step 1: Loading the Transformers ASR Model

To begin building our speech recognition demo, we first need to have an Automatic Speech Recognition (ASR) model. You can either train your own model or use a pre-trained one. In this tutorial, we will leverage a pre-trained ASR model called "whisper" from OpenAI.

Loading the "whisper" model from the Hugging Face Transformers library is a straightforward process. Here's the code snippet to accomplish this:

```python
from transformers import pipeline
p = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")
```

With just these two lines of code, we initialize a pipeline for automatic speech recognition using the "openai/whisper-base.en" model. The pipeline abstracts away the complexities of working with the model directly, providing a high-level interface for performing ASR tasks.

By utilizing a pre-trained model like "whisper", we can quickly get started with building our demo without the need for extensive model training. This allows us to focus on integrating the model into our application and creating an engaging user experience.

## Step 2: Building a Full-Context ASR Demo with Transformers

Our first step in creating the speech recognition demo is to build a *full-context* ASR demo. In this demo, the user will speak the entire audio before the ASR model processes it and generates the transcription. Thanks to Gradio's intuitive interface, building this demo is a breeze.

We'll start by creating a function that wraps around the `pipeline` object we initialized earlier. This function will serve as the core of our demo, handling the audio input and generating the transcription.

To capture the user's audio input, we'll utilize Gradio's built-in `Audio` component. This component will be configured to accept input from the user's microphone and return the filepath of the recorded audio. For displaying the transcribed text, we'll use a simple `Textbox` component.

The `transcribe` function, which is the heart of our demo, takes a single parameter called `audio`. This parameter represents the audio data recorded by the user, stored as a NumPy array. However, the `pipeline` object expects the audio data to be in the `float32` format. To ensure compatibility, we first convert the audio data to `float32` and then normalize it by dividing it by its maximum absolute value. Finally, we pass the processed audio data to the `pipeline` object to obtain the transcribed text.

In [8]:
!ls -lah ~/.cache/huggingface/hub

total 0
drwxr-xr-x. 5 1000740000 root 128 Jan 13 10:20 .
drwxr-xr-x. 4 1000740000 root  67 Jan 13 10:19 ..
drwxr-xr-x. 6 1000740000 root  65 Jan 13 10:18 datasets--hf-internal-testing--librispeech_asr_dummy
drwxr-xr-x. 4 1000740000 root 114 Jan 13 10:20 .locks
drwxr-xr-x. 6 1000740000 root  65 Jan 13 10:20 models--distil-whisper--distil-medium.en


In [9]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained(model_id.value)

pt_model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id.value)
pt_model.eval();

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/opt/app-root/lib64/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/app-root/lib64/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue:

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

In [10]:
!ls -lah ~/.cache/huggingface/hub

total 4.0K
drwxr-xr-x. 6 1000740000 root 187 Jan 13 10:35 .
drwxr-xr-x. 4 1000740000 root  67 Jan 13 10:19 ..
drwxr-xr-x. 6 1000740000 root  65 Jan 13 10:18 datasets--hf-internal-testing--librispeech_asr_dummy
drwxr-xr-x. 5 1000740000 root 154 Jan 13 10:35 .locks
drwxr-xr-x. 6 1000740000 root  65 Jan 13 10:20 models--distil-whisper--distil-medium.en
drwxr-xr-x. 6 1000740000 root  65 Jan 13 10:37 models--openai--whisper-large-v2
-rw-r--r--. 1 1000740000 root   1 Jan 13 10:35 version.txt


In [11]:
import torch
this_device = "cuda" if torch.cuda.is_available() else "cpu"
this_device

'cpu'

In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [13]:
# aa: comment - pytorch doesnt have the tool

### Preparing the Input Sample

To use the Whisper model for speech recognition, we need to properly prepare the input audio sample. The `WhisperProcessor` expects the audio data to be in the form of a NumPy array, along with information about the audio sampling rate. It then processes the audio and returns the `input_features` tensor, which is used for making predictions.

The conversion of the audio file to the required NumPy format is conveniently handled by the Hugging Face Datasets library. This library provides a seamless interface for loading and preprocessing audio data, making it easier to integrate with the Whisper model.

To prepare the input sample, the next Python code:

1. Loads the audio file using the Hugging Face Datasets library.
2. Extracts the audio data as a NumPy array and obtain the sampling rate.
3. Passes the audio array and sampling rate to the `WhisperProcessor`.
4. Retrieves the `input_features` tensor from the processor.

In [15]:
!ls -alh  ~/.cache/huggingface/datasets

total 4.0K
drwxr-xr-x. 4 1000740000 root 4.0K Jan 13 10:38 .
drwxr-xr-x. 4 1000740000 root   67 Jan 13 10:19 ..
drwxr-xr-x. 3 1000740000 root   19 Jan 13 10:38 distil-whisper___librispeech_long
drwxr-xr-x. 3 1000740000 root   19 Jan 13 10:13 hf-internal-testing___librispeech_asr_dummy
-rw-r--r--. 1 1000740000 root    0 Jan 13 10:38 _opt_app-root_src_.cache_huggingface_datasets_distil-whisper___librispeech_long_clean_0.0.0_164d3b41852b1eebe89f1dc0e6e0042f16835ea0.lock
-rw-r--r--. 1 1000740000 root    0 Jan 13 10:18 _opt_app-root_src_.cache_huggingface_datasets_hf-internal-testing___librispeech_asr_dummy_clean_0.0.0_5be91486e11a2d616f4ec5db8d3fd248585ac07a.lock


In [ ]:
from datasets import load_dataset

def extract_input_features(sample):
    input_features = processor(
        sample["audio"]["array"],
        sampling_rate=sample["audio"]["sampling_rate"],
        return_tensors="pt",
    ).input_features
    return input_features

dataset = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy", "clean", split="validation"
)
sample = dataset[0]
input_features = extract_input_features(sample)

In [ ]:
!ls -alh  ~/.cache/huggingface/datasets

### Running Model Inference

With the input sample prepared, we can now perform speech recognition using the Whisper model. The model provides a convenient `generate` interface that simplifies the inference process. Here's how you can run the model inference:

1. Pass the `input_features` tensor to the `generate` method of the Whisper model.
2. The model will process the input and generate the predicted token IDs.
3. Once the generation is complete, use the `processor.batch_decode` method to decode the predicted token IDs into human-readable text transcription.

The `generate` method handles the complex task of sequence generation, taking into account the model's architecture and the provided input features. It produces the predicted token IDs, which represent the transcribed text in a encoded format.

By leveraging the `generate` interface and the `processor.batch_decode` method, you can easily perform speech recognition with the Whisper model. The model takes care of the complex task of mapping the audio input to text output, while the processor handles the necessary decoding step to provide you with the final transcription.

In [ ]:
%%time
import IPython.display as ipd

predicted_ids = pt_model.generate(input_features)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

display(ipd.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"]))
print(f"Reference: {sample['text']}")
print(f"Result: {transcription[0]}")